CNN(Convolutional Neural Network, 합성곱 신경망) 
: 딥러닝 모델 (ex. 이미지/자연어 분류)

1.합성곱(convolution) : filter = mask = window = kernel 로 불리는 특정 크기의 행렬을 이미지 or 문장 데이터 행렬에 슬라이딩하면서 곱하고 더하는 연산.
  * stride : filter 위치를 몇 칸 이동할지 결정하는 값 
  * 슬라이딩 : filter가 이미지 데이터 행렬 위를 상하좌우로 이동하는 동작
  * filter가 더 이상 슬라이딩 할 수 없을 때까지 반복
  * 특징맵(feature map) : 최종 결과 / 합성곱 연산을 거칠 때마다 작아진다. -> 방지 : padding 사용
  * padding : 출력 크기 조정 / 0으로 채움


2.풀링 연산 : 특징맵의 크기 줄임, 주요한 특징 추출
  - 최대 풀링(max pooling)多 / 평균 풀링(average pooling)


In [37]:
# To Do! Q(질문)를 label(감정)별로 분류 (cf. A(답변) 사용 X)
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

train_file = './data/chatbot_data.csv'
data = pd.read_csv(train_file, delimiter=',')
print(data)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [38]:
features = data['Q'].tolist()
labels = data['label'].tolist()

corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
print(corpus[:5])

[['12시', '땡'], ['1지망', '학교', '떨어졌어'], ['3박4일', '놀러가고', '싶다'], ['3박4일', '정도', '놀러가고', '싶다'], ['ppl', '심하네']]


In [39]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print(sequences[:5])

[[4646, 4647], [4648, 343, 448], [2580, 803, 11], [2580, 804, 803, 11], [4649, 2581]]


In [40]:
word_index = tokenizer.word_index
# print(word_index)

In [41]:
# 전체 벡터 크기를 동일하게 맞춰야 함
MAX_SEQ_LEN = 15 
padded_seqs = preprocessing.sequence.pad_sequences(sequences, 
                                                   maxlen=MAX_SEQ_LEN, # 시퀀스의 최대 길이
                                                   padding='post') # 빈 공간을 0으로 채우는 작업
print(padded_seqs)                                                   

[[ 4646  4647     0 ...     0     0     0]
 [ 4648   343   448 ...     0     0     0]
 [ 2580   803    11 ...     0     0     0]
 ...
 [13395  2517    89 ...     0     0     0]
 [  147    46    91 ...     0     0     0]
 [  555 13398     0 ...     0     0     0]]


In [42]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

# train:valid:test = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼파라미터
dropout_prob = 0.5
EMB_SIZE = 128 # 임베딩 결과로 나올 밀집 벡터 크기
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

# 단어 임베딩
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer) # 오버피팅(과적합)에 대비

# 특징 추출 (특징맵)
# 합성곱 연산
conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1) # 최대 풀링 연산

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1, pool2, pool3])

# 완전 연결 계층
hidden = Dense(128, # 출력 노드 수
               activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)

logits = Dense(3, # 3가지 클래스로 감정 분류 해야 하기 때문에
               name='logits')(dropout_hidden)

# 감정 클래스별 확률 
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# CNN 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # 손실 함수
              metrics=['accuracy'])

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH
          , verbose=1) # 진행 과정 상세히 (<-> 0:생략)

Epoch 1/5
414/414 [==============================] - 17s 38ms/step - loss: 0.9041 - accuracy: 0.5677 - val_loss: 0.6645 - val_accuracy: 0.7356
Epoch 2/5
414/414 [==============================] - 15s 36ms/step - loss: 0.5525 - accuracy: 0.7826 - val_loss: 0.3056 - val_accuracy: 0.9023
Epoch 3/5
414/414 [==============================] - 14s 35ms/step - loss: 0.3118 - accuracy: 0.8948 - val_loss: 0.1525 - val_accuracy: 0.9526
Epoch 4/5
414/414 [==============================] - 13s 31ms/step - loss: 0.1951 - accuracy: 0.9378 - val_loss: 0.0932 - val_accuracy: 0.9670
Epoch 5/5
414/414 [==============================] - 14s 33ms/step - loss: 0.1412 - accuracy: 0.9564 - val_loss: 0.0645 - val_accuracy: 0.9793


In [43]:
loss, accuracy = model.evaluate(test_ds, verbose=1)

60/60 [==============================] - 0s 4ms/step - loss: 0.0709 - accuracy: 0.9763


In [44]:
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

Accuracy: 97.631133
loss: 0.070869


In [45]:
model.save('cnn_model.h5')

In [46]:
test_ds = ds.take(2000).batch(20)

model = load_model('cnn_model.h5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 128)      1715072     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 15, 128)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 13, 128)      49280       dropout_2[0][0]                  
____________________________________________________________________________________________

In [47]:
model.evaluate(test_ds, verbose=2)

100/100 - 1s - loss: 0.0572 - accuracy: 0.9845


[0.057181622833013535, 0.984499990940094]

In [51]:
print('단어 시퀀스 :', corpus[10212])
print('단어 인덱스 시퀀스 :', padded_seqs[10212])
print('문장 분류(정답) :', labels[10212])

단어 시퀀스 : ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 : [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답) : 2


In [52]:
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print('감정 예측 점수 :', predict)
print('감정 예측 클래스 :', predict_class.numpy())

감정 예측 점수 : [[2.5848356e-06 2.1340316e-05 9.9997604e-01]]
감정 예측 클래스 : [2]
